# Model development

---

**Note:** This code was used to investigate the data and identify the best preprocessing steps and the best model for the data. Therefore the code isvery messy and exploratory in nature. The final code used when actually running the preprocessing and the model is cleansed code based on the knowledge gained from this exploration.

I have included comments in this file to explain what I did and hopefully give insight into the process of investigation that I followed.

## Summary

**Goal**

The purpose of this project is to classify invoices into one of approximately 20 different accounting classifications based on information from the face of the invoice, such as the invoice description, the supplier company name, the amount of invoice and the currency.

This is a real world problem encountered by a number of companies. This project is a proof of concept for a wider, more in depth project to potentially develop a more general model and, assuming it provides sufficiently good results, implement it into production environments.

**Metric(s)**

As this is a classification problem with the focus being on accurate classification of invoices into the relevant categories (with limited negative effects for incorrect classification) the key metrics for this project are: 
- the accuracy score and 
- the area under the receiver operating characteristic curve (AUC-ROC).

**Data**

The data has been sourced from a company's accounting systems and is real data that, unfortunately, I cannot share. A data dictionary has been provided below.

**Findings**

It is possible to predict the correct invoice classification using data from the face of the invoice with an accuracy of approximately **94.4%** achieved on the test data set.

**Limitations**

The data available has 2.18 million rows and, due to data sharing restrictions, all processing and analysis had to be completed on my personal laptop. In order to work with the data in a reasonable time frame with the limited computing power, I worked with a subset of the data, created by selecting those invoices in GBP with no null values in the required columns.

Therefore the resulting model is specific to those invoices in GBP (which are likely to have an invoice description in English) which have data in all required columns.

Additional work will need to be completed to generalise the model to non-GBP invoices and also to be able to handle null values.


## Model walk through

This section covers the entire scope of the project, from data collection through exploratory data analysis (EDA), data cleansing and processing through to modelling and evaluation.

### Data Collection

The data has been collected from a number of different accounting systems owned by the global group of companies. This data is being used for a number of different purposes so there has been an amount of pre-processing and cleansing that had already happened to the data before I have access to it. This included:

- Combining multiple import tables from different systems into a single table
- Standardising the different columns (including the classification groups - the target)) across the different source tables
- Adding the local currency conversions
- Deleting records where certain criteria is met
    
To access the data from the Microsoft SQL Server database I used the following code:

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import credentials # A file containing the required credentials for accessing the database

# Set jupyter notebook settings to be able to view up to 999 columns of a pandas dataframe without concatenation
pd.options.display.max_columns = 999

# Set up variables for database access
db_driver = credentials.database_driver
db_server = credentials.database_server
db_database = credentials.database
sql_schema = credentials.schema
sql_table = credentials.table

# Set up the connection to the database
db_creds = 'DRIVER={'+db_driver+'}; SERVER='+db_server+'; DATABASE='+db_database+';Trusted_Connection=yes;'
cnxn = pyodbc.connect(db_creds)

# Create query for selecting columns from required table, filtering for non-intercompany rows
sql_columns_filtered = "[Invoice Operating Unit],[Business Unit],[Service Line],[Region],[OU/Country]\
,[Supplier Number],[Supplier Name],[Invoice Type],[Invoice Date],[Paid Date],[Invoice Desc],[PO Number]\
,[Invoice_Amt],[Invoice Currency],[USD_Amt],[V Site Country],[Pay Group],[Payment Status],[Project Owning Org]\
,[Project Number],[Country],[datafile],[datasource],[Legacy],[Year],[Contract_type],[Customer],[Service_line]\
,[Funding_structure],[Category_Group],[Supplier_Group],[Supplier_Sector],[Leakage_Identifier],[Leakage_Group]\
,[Serv_Mtrl],[Reimbursable_Flag],[Intercompany_Flag],[Americas_Flag],[Reporting_Date]"

sql_query_filtered = 'select '+sql_columns_filtered+' from '+sql_schema+'.'+sql_table\
+" where [Intercompany_Flag] = 'Non-intercompany'"

# Read the data from the database into a pandas dataframe
data_filtered = pd.read_sql(sql_query_filtered,cnxn)

# Save the data to a local .csv file
data_filtered.to_csv('data_2018_12_07.csv',encoding='utf-8',index=False)

I saved the data to a csv file locally so I had a fixed baseline to work with that wouldn't change if the database were updated. A data dictionary is provided below.

### EDA

I started by looking at the columns available in the data and understanding what type of information the columns were supposed to contain. I created a data dictionary to record this information (below).

To start my exploratory data analysis (EDA) I looked at the data in first few rows to see if the data matched the understanding based on the data dictionary. In addition I set the data types for each column and set to null those entries that didn't match the expected data type.

In [3]:
import pandas as pd
import numpy as np

# Set all data types to string for initial import
col_dtypes = {'Invoice Operating Unit':str, 'Business Unit':str, 'Service Line':str, 'Region':str,
       'OU/Country':str, 'Supplier Number':str, 'Supplier Name':str, 'Invoice Type':str,
       'Invoice Date':str, 'Paid Date':str, 'Invoice Desc':str, 'PO Number':str, 'Invoice_Amt':str,
       'Invoice Currency':str, 'USD_Amt':float, 'V Site Country':str, 'Pay Group':str,
       'Payment Status':str, 'Project Owning Org':str, 'Project Number':str, 'Country':str,
       'datafile':str, 'datasource':str, 'Legacy':str, 'Year':str, 'Contract_type':str, 'Customer':str,
       'Service_line':str, 'Funding_structure':str, 'Category_Group':str, 'Supplier_Group':str,
       'Supplier_Sector':str, 'Leakage_Identifier':str, 'Leakage_Group':str, 'Serv_Mtrl':str,
       'Reimbursable_Flag':str, 'Intercompany_Flag':str, 'Americas_Flag':str,
       'Reporting_Date':str}

# Import the data from the .csv file
data = pd.read_csv('data_2018_12_07.csv',na_values='Unknown',dtype=col_dtypes)

#Check the head of the data
data.head()

# Set the data types of the relevant columns to the correct data type
data['Invoice Date'] = pd.to_datetime(data['Invoice Date'],errors='coerce')
data['Paid Date'] = pd.to_datetime(data['Paid Date'],errors='coerce')
data['Reporting_Date'] = pd.to_datetime(data['Reporting_Date'],errors='coerce')
data['Supplier Number'] = pd.to_numeric(data['Supplier Number'],errors='coerce')
data['Invoice_Amt'] = pd.to_numeric(data['Invoice_Amt'],errors='coerce')
data['USD_Amt'] = pd.to_numeric(data['USD_Amt'],errors='coerce')
data['Year'] = pd.to_numeric(data['Year'],errors='coerce')

I then looked at summary information such as the total number of records, the number of nulls by column and the number of unique elements per column.

In [3]:
# Check the shape of the data
data.shape
# Output: (2189333, 39)

# Percentage of each column that is null
data.isnull().sum()/len(data)

My next step was to visualise the data in Tableau to understand a bit more what the data contained and quickly interrogate and investigate what was available. One of the views I created was the ability to select a column and view the different elements available in that column, the count of those elements and whether they were nulls, unclassified, blank, zero or otherwise.

In [1]:
# Tableau workbook not available due to data sharing restrictions

The two most important columns are the target (invoice classification) and invoice description (as it is likely to be a strong predictor of the target). I reviewed these two columns in more detail including: reviewing the class imbalance and identifying the number of unclassified invoices for the target; and looking at a random sample of the invoice descriptions to understand what data they contain and whether it was as expected.

In [ ]:
# Understand class imbalance
data['Category_Group'].value_counts()

# Output:

# Category 1     637855
# Category 2     439340    NB: This is the 'Unclassified' category
# Category 3     206532
# Category 4     185722
# Category 5     106766
# Category 6     96262
# Category 7     69142
# Category 8     64971
# Category 9     63096
# Category 10    60642
# Category 11    59679
# Category 12    45523
# Category 13    39289
# Category 14    31918
# Category 15    20553
# Category 16    20341
# Category 17    14289
# Category 18    7945
# Category 19    7242
# Category 20    639
# Category 21    626
# Category 22    268
# Category 23    114
# Category 24    51
# Category 25    50
# Category 26    41
# Category 27    16
# Category 28    15
# Category 29    13
# Category 30    12
# Category 31    11
# Category 32    7
# Category 33    5
# Category 34    4
# Category 35    3
# Category 36    3

# Create random sample for review
# NB: there is no no random state defined so each time it is run it provides a different sample
data.sample(20)

### Data Cleansing

The complete data set contains over 2.18 million rows of data from a wide variety of source systems, countries and languages. As such there are a large number of nulls, multiple languages, multiple character sets in text data, string values in numerical columns and category groups like 'Unclassified', 'Intercompany' and 'Do Not Use'.

Due to the restrictions on where I could host the commercially sensitive data, I would have to work purely from my laptop (rather than using cloud based services like AWS). Therefore, in order to be able to process the data in a reasonable time period I had to select a subset of the data to work with.

(It took a lot of trial and error to work out the size of the dataset that would work best - giving the ability to work on my laptop whilst still retaining enough data to make the model a true proof of concept for further development for the wider data set.)

When creating this subset I considered:
- future natural language processing would be significantly easier if the data were all in the same language
- as category group is the target variable, it made sense to exclude those category groups that are unwanted
- some columns had very high levels of null values, so it made sense to exclude them
- there is very high class imbalance, so reducing the size of the data at this stage would help keep the training data set small (even after any oversampling to redress the imbalance)

Therefore I:
- dropped columns with high levels of null values, 
- dropped rows with unwanted category groups (the target)
- dropped remaining rows with any null values, and 
- only selected those invoices with a local currency of GBP.

This last point was on the assumption that an invoice in GBP was likely to have an English description.

**NB:** I did attempt to identify the invoice description language row by row using the Python library 'langdetect'. However, due to the high level of technical language and the low numbers of words in the invoice descriptions, this approach did not return any sensible results. For example, 'WATER 10L' was identified as German, even though the invoice was in GBP and the supplier was a British company.

**Data cleansing**

In [4]:
import pandas as pd
import numpy as np

# Drop rows for 'UNCLASSIFIED','INTERCOMPANY','DONOTUSE-SUBCONTRACTOR'
data_processing = data[~data['Category_Group'].isin(['UNCLASSIFIED','INTERCOMPANY','DONOTUSE-SUBCONTRACTOR'])]

# Keep columns with low percentage of null values
column_mask = ['Business Unit', 'Supplier Number', 'Supplier Name', 
       'Invoice Date', 'Invoice Desc', 'Invoice_Amt',
       'Invoice Currency', 'USD_Amt', 
       'Project Owning Org', 
       'datasource', 'Legacy', 'Year', 
       'Category_Group', 'Supplier_Group',
       'Leakage_Identifier', 'Leakage_Group', 'Intercompany_Flag', 'Americas_Flag',
       'Reporting_Date']

data_processing = data_processing[column_mask].copy()

# Drop remaining nulls
data_processing.dropna(inplace=True)

# Retain only invoices in GBP
data_processing_english = data_processing[data_processing['Invoice Currency'].isin(['GBP'])]

# Remaining records
data_processing_english.shape
# Output:
# (391410, 19)

# Save to .csv
data_processing_english.to_csv('Cleansed data_GBP.csv',encoding='utf-8',index=False)

**Attempted language detection** (not used)

In [6]:
from langdetect import detect, DetectorFactory
from datetime import datetime

# Sub set the data for testing purposes
data_processing_1 = data_processing_english.iloc[0:10000].copy()

# Set a seed so the results are consistent each time it is run
# The algorithm is probabilistic based and, for low word counts, different results are obtained for different seeds
DetectorFactory.seed = 0

# Initiate a timer
startTime = datetime.now()

output_1 = []
count = 0
for text in data_processing_1['Invoice Desc']:
    # Try except statement captures those instances where the algorithm is unable to classify the language
    try:
        out = detect(text)
    except:
        out = 'n/a'
    output_1.append(out)
    # A counter to show progress
    count+=1
    if count%1000 == 0:
        print(count)
        
# Print time taken to run
print(datetime.now() - startTime)

# Add the output to an additional column in the dataframe for review purposes
data_processing_1['language'] = output_1

### Data Processing

The data processing stage is where ther majority of the time and effort was spent, attempting to identify which elements of the data provided the best predictive ability.

The first step was to split the data into a train and test set, with the test data then able to be used later to confirm how well the model performed.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('Cleansed data_GBP.csv',na_values='Unknown')

X = data.copy()
y = X.pop('Category_Group')

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, stratify=y)

The pre-processing steps are different for the train and the test sets, as the training data is used to fit the data conversion steps whereas the test data is purely converted and then used in the model to generate predictions.

### Training data

The first step for the training data is to start to rebalance the classes. This step is split into three elements:

1. remove those classes with very few counts
2. randomly undersample those classes with very high counts
3. use a k-nearest-neighbours approach to generate more synthetic counts for those classes with low counts

As step three uses a KNN model to generate the synthetic points, the data needs to be in a format suitable for modelling. Therefore, this is done as the final preprocessing step prior to modelling.

**Rebalance classes - Step 1 - removing classes with very few counts**

This step simply removes those classes which have very low counts. This means these classes will never be predicted by the model but as they are so small, the trade off should be minimal in terms of overall accuracy.

In [ ]:
# Value counts for each category
data_value_counts = pd.DataFrame(y_train.value_counts(dropna=False))

# Create a list of those categories with counts > 100
categories_not_low = data_value_counts[data_value_counts['Category_Group']>100].index

# Subset y_train for those categories that are not low
y_train = y_train[y_train.isin(categories_not_low)]

# Subset X_train for the indexes in y_train
X_train = X_train.loc[y_train.index]

**Rebalance classes - Step 2 - random undersampling**

This step splits the data into two sections: the first with those classes with high counts per category; and the second the remainder.

The first set is the randomly undersampled to generate a data set with equal class counts.

This undersampled first set is then combined with the second 'remainder' set to recreate the X_train and y_train sets but with those large categories undersampled.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# Create a list of those categories with counts > 10,000
categories_too_high = data_value_counts[data_value_counts['Category_Group']>10000].index

# Create a subset of y_train and X_train filtered for those high count categories
y_train_too_high = y_train[y_train.isin(categories_too_high)].copy()
X_train_too_high = X_train.loc[y_train_too_high.index].copy()

# Create a subset of y_train and X_train excluding those high count categories
y_train_remainder = y_train[~y_train.isin(categories_too_high)].copy()
X_train_remainder = X_train.loc[y_train_remainder.index].copy()

# Instantiate the Random Under Sampler
under_sampler = RandomUnderSampler()

# Under sample the high count category subsets
X_train_undersampled, y_train_undersampled = under_sampler.fit_sample(X_train_too_high,y_train_too_high)

# Convert the output back to a pandas format and append to the remainder sets
y_train_undersampled = pd.Series(y_train_undersampled)
y_train = y_train_undersampled.append(y_train_remainder,ignore_index=True)

X_train_undersampled = pd.DataFrame(X_train_undersampled,columns=X_train.columns)
X_train = X_train_undersampled.append(X_train_remainder,ignore_index=True)

**Cleaning text**

After viewing a random sample of the text data, there were some specific items that needed to be resolved, as well as applying some standard natural language processing steps.

The steps are:
- remove specific characters (\r, \n)
- expand linguistic contractions (e.g. don't => do not)
- tokenise the text strings
- remove non-ascii characters
- convert all text to lowercase
- remove punctuation
- obtain the stem for each word

Each step contains a trade off in understanding of text. For example, converting text to lowercase removes information about words that start a sentence, or are a name rather than another word (e.g. Apple the company and apple the fruit). However, after running a basic decision tree model on the output for each combination of processing steps, this combination of processing steps seemed to provide the best.

In addition, I tried replacing numerical values with their string equivalent (e.g. 2 becomes two) but this did not seem to improve the quality of predictions.

In [ ]:
import contractions, unicodedata, re
from nltk.stem import LancasterStemmer

X_train = X_train.copy()

# Replace specific characters
replace_dict = {
    '\r':' ',
    '\n':' '
}

for key,value in replace_dict.items():
    X_train['Invoice Desc'] = [text.replace(key,value) for text in X_train['Invoice Desc']]
    
# Fix contractions (e.g. can't => cannot)
X_train['Invoice Desc'] = [contractions.fix(text) for text in X_train['Invoice Desc']]

# Tokenise
X_train['Invoice Desc'] = [nltk.word_tokenize(text) for text in X_train['Invoice Desc']]


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

# Run functions above to remove non-ascii characters, 
# convert to lowercase, remove punctuation and obtain the word stems
t0 = time.time()
X_train['Invoice Desc'] = [remove_non_ascii(text) for text in X_train['Invoice Desc']]
t1 = time.time()
print('Non ascii: ',t1-t0)
X_train['Invoice Desc'] = [to_lowercase(text) for text in X_train['Invoice Desc']]
t2 = time.time()
print('Lowercase: ',t2-t1)
X_train['Invoice Desc'] = [remove_punctuation(text) for text in X_train['Invoice Desc']]
t3 = time.time() 
print('Remove punctuation: ',t3-t2)
t6 = time.time()
X_train['Invoice Desc'] = [stem_words(text) for text in X_train['Invoice Desc']]
t7 = time.time()
print('Stem words: ',t7-t6)

# Reconstruct description after tokenisation for use in CountVectorizer
X_train['Invoice Desc'] = [' '.join(text) for text in X_train['Invoice Desc']]

This is the function for replacing numbers with their string equivalent, which wasn't used in the final model

In [ ]:
def replace_numbers(words):
    """Replace all integer occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

# Run replace_numbers function (not used)
t4 = time.time()
X_train['Invoice Desc'] = [replace_numbers(text) for text in X_train['Invoice Desc']]
t5 = time.time()
print('Stem words: ',t5-t4)

**Vectorising text**

The next step was to vectorise the cleansed text using Count Vectorizer, as well as other columns including the supplier name, supplier group and internal project name. 

The choice to vectorise the supplier and project information was based on the theory that the names may reflect the type of industry or project and hence the potential classification that the invoices linked with them.

I also tried simply dummifying the supplier and project columns (rather tha vectorising them) but the results were less good.

The CountVectorizer returns a sparse matrix and, due to the size of the data and memory limitations from the laptop, I chose to keep the output in this format and also convert all the other data to sparse matrices and combine them, instead of trying to work with pandas dataframes.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvec_supplier = CountVectorizer(token_pattern='\w+',stop_words='english',max_df=1.0,min_df=10)
cvec_invoice = CountVectorizer(token_pattern='\w+',stop_words='english',max_df=1.0,min_df=10)
cvec_currency = CountVectorizer(token_pattern='\w+',stop_words='english',max_df=1.0,min_df=10)
cvec_project = CountVectorizer(token_pattern='\w+',stop_words='english',max_df=1.0,min_df=10)
cvec_group = CountVectorizer(token_pattern='\w+',stop_words='english',max_df=1.0,min_df=10)

supplier_name_nlp = X_train['Supplier Name']
invoice_desc_nlp = X_train['Invoice Desc']
currency_nlp = X_train['Invoice Currency']
project_nlp = X_train['Project Owning Org']
group_nlp = X_train['Supplier_Group']

cvec_supplier.fit(supplier_name_nlp)
cvec_invoice.fit(invoice_desc_nlp)
cvec_currency.fit(currency_nlp)
cvec_project.fit(project_nlp)
cvec_group.fit(group_nlp)

supplier_sparse_matrix = cvec_supplier.transform(supplier_name_nlp)
invoice_sparse_matrix = cvec_invoice.transform(invoice_desc_nlp)
currency_sparse_matrix = cvec_currency.transform(currency_nlp)
project_sparse_matrix = cvec_project.transform(project_nlp)
group_sparse_matrix = cvec_group.transform(group_nlp)

**Dummifying columns**

The remaining columns containing data in string format were dummified using one-hot-encoding in pandas, then converted to a sparse matrix.

In [ ]:
from scipy import sparse

dumm_cols = ['Business Unit','datasource','Legacy','Leakage_Identifier','Leakage_Group','Americas_Flag']

dummified_data = pd.get_dummies(X_train[dumm_cols],drop_first=True)
dummified_sparse_matrix = sparse.csr_matrix(dummified_data)

**Remaining columns**

The remaining columns were converted to numerical values, then converted to a sparse matrix.

In [ ]:
retain_cols = ['Invoice_Amt','USD_Amt','Year']
retain_data = pd.DataFrame()

for col in retain_cols:
    retain_data[col] = pd.to_numeric(X_train[col])
    
retain_sparse_matrix = sparse.csr_matrix(retain_data)

**Combine sparse matrices**

The next step for the training data was to combine the individual sparse matrices into a single matrix, ready for using in the oversampling stage.

In [ ]:
sparse_matrices = [
    supplier_sparse_matrix,
    invoice_sparse_matrix,
    currency_sparse_matrix,
    project_sparse_matrix,
    group_sparse_matrix,
    dummified_sparse_matrix,
    retain_sparse_matrix
]

X_train_sparse = sparse.hstack(sparse_matrices)

**Feature names**

A list of the feature names is generated (for use during the model evaluation).

In [ ]:
feature_names = cvec_supplier.get_feature_names()\
                +cvec_invoice.get_feature_names()\
                +cvec_currency.get_feature_names()\
                +cvec_project.get_feature_names()\
                +cvec_group.get_feature_names()\
                +list(dummified_data.columns)\
                +list(retain_data.columns)

**Rebalance classes - Step 3 - SMOTE oversampling**

The final step for the training data was to use the SMOTE oversampling method to increase the count of those classes with few entries by creating synthetic values based on a K-Nearest-Neighbours approach. After this step then all the classes would be balanced and the data would be ready for modelling.

In [ ]:
from imblearn.over_sampling import SMOTE

sampler = SMOTE(random_state=1,n_jobs=-1)
X_resampled, y_resampled = sampler.fit_sample(X_train_sparse, y_train)

**Save training data to .csv**

The final stage for the training data was to save it to .csv for use in the modelling stage. This meant the processing and oversampling did not need to be run each time a new model was tested.

In [ ]:
sparse.save_npz('X train_processed data_GBP.npz',X_resampled)
pd.DataFrame(y_resampled).to_csv('y train_processed data_GBP.csv')
pd.Series(feature_names).to_csv('X train_processed data_GBP_feature names.csv')

### Test data and training data without balancing the classes

In order to have a test set against which the trained model could be tested, a similar set of processing steps has to be applied to the training data. In addition, applying the same steps to the training data but without applying the rebalancing steps allows a comparison between training scores and test scores, and helps to determine how well the model generalises.

**NB:** it is not possible to compare the rebalanced training scores against the test scores as the baselines are significantly different.

The same process is applied to both the test data and the imbalanced training data. The code below is for the test data but it is identical for both.

The code replicates the steps applied for the training data but does not rebalance the classes and only transforms the data (rather than refitting the vectoriser). In addition, the dummification of columns required ensuring the resulting columns matched with the dummified balanced training data (see code below).

The data is retained as a sparse matrix as for the training data.

In [ ]:
X_test = X_test.copy()

# Replace specific characters
for key,value in replace_dict.items():
    X_test['Invoice Desc'] = [text.replace(key,value) for text in X_test['Invoice Desc']]
    
# Fix contractions (e.g. can't => cannot)
X_test['Invoice Desc'] = [contractions.fix(text) for text in X_test['Invoice Desc']]

# Tokenise
X_test['Invoice Desc'] = [nltk.word_tokenize(text) for text in X_test['Invoice Desc']]

# Remove non-ascii characters, convert to lowercase and remove punctuation
t0 = time.time()
X_test['Invoice Desc'] = [remove_non_ascii(text) for text in X_test['Invoice Desc']]
t1 = time.time()
print('Non ascii: ',t1-t0)
X_test['Invoice Desc'] = [to_lowercase(text) for text in X_test['Invoice Desc']]
t2 = time.time()
print('Lowercase: ',t2-t1)
X_test['Invoice Desc'] = [remove_punctuation(text) for text in X_test['Invoice Desc']]
t3 = time.time()
print('Remove punctuation: ',t3-t2)

# Stem the words
t6 = time.time()
X_test['Invoice Desc'] = [stem_words(text) for text in X_test['Invoice Desc']]
t7 = time.time()
print('Stem words: ',t7-t6)

# Reconstruct description after tokenisation
X_test['Invoice Desc'] = [' '.join(text) for text in X_test['Invoice Desc']]

This step applies the existing fitted CountVectorizer to transform the test data

In [ ]:
# Vectorise using fitted CountVectorizer from training data
supplier_name_nlp = X_test['Supplier Name']
invoice_desc_nlp = X_test['Invoice Desc']
currency_nlp = X_test['Invoice Currency']
project_nlp = X_test['Project Owning Org']
group_nlp = X_test['Supplier_Group']

supplier_sparse_matrix = cvec_supplier.transform(supplier_name_nlp)
invoice_sparse_matrix = cvec_invoice.transform(invoice_desc_nlp)
currency_sparse_matrix = cvec_currency.transform(currency_nlp)
project_sparse_matrix = cvec_project.transform(project_nlp)
group_sparse_matrix = cvec_group.transform(group_nlp)

This step dummifies the test data, then compares the columns generated by dummification between the test and the training data. Any columns that match are kept. Any columns in the test set but not in the train set are dropped and any columns in the train set but not in the test set are created and set to zero.

The data is the converted to a sparse matrix.

In [ ]:
# Dummify test data
dummified_data_test = pd.get_dummies(X_test[dumm_cols],drop_first=False)

# Filter out dummified columns not in training data
test_col_in_train = list(compress(dummified_data_test.columns, dummified_data_test.columns.isin(dummified_data.columns)))
dummified_data_test = dummified_data_test[test_col_in_train]

# Add columns in dummified train set that are not in dummified test set
train_col_not_in_test = list(set(dummified_data.columns)-set(dummified_data_test.columns))
for col in train_col_not_in_test:
    dummified_data_test[col] = 0
    
# Convert to sparse matrix
dummified_sparse_matrix = sparse.csr_matrix(dummified_data_test)

The remaining (numerical) columns are converted to numeric format and then converted to a sparse matrix.

In [ ]:
# Convert numeric columns to numeric data type
retain_data = pd.DataFrame()
for col in retain_cols:
    retain_data[col] = pd.to_numeric(X_test[col])
    
# Convert to sparse matrix
retain_sparse_matrix = sparse.csr_matrix(retain_data)

This step combines the sparse matrices together to greate the test data.

In [ ]:
sparse_matrices = [
    supplier_sparse_matrix,
    invoice_sparse_matrix,
    currency_sparse_matrix,
    project_sparse_matrix,
    group_sparse_matrix,
    dummified_sparse_matrix,
    retain_sparse_matrix
]

X_test_sparse = sparse.hstack(sparse_matrices)

The data is then saved to .csv.

In [ ]:
sparse.save_npz('X test_processed data_GBP.npz',X_test_sparse)
pd.DataFrame(y_test).to_csv('y test_processed data_GBP.csv')

### Modelling

**Import data and calculate baselines**

The first step is to import the data and to calculate the baseline values for each data set. The baseline shows the accurancy score that would be achieved purely by predicting the most common occuring value as the prediction for all inputs.

The baseline for the balanced training data is significantly below the unbalanced training data, as expected.

In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse

# Import balanced training, unbalanced training and test data
X_train = sparse.load_npz('X train_processed data_GBP.npz')
y_train = pd.read_csv('y train_processed data_GBP.csv')
y_train.columns = ['index','y']
y_train = y_train['y']
X_test = sparse.load_npz('X test_processed data_GBP.npz')
y_test = pd.read_csv('y test_processed data_GBP.csv')
y_test.columns = ['index','y']
y_test = y_test['y']
X_train_imbalanced = sparse.load_npz('X test_processed data_GBP.npz')
y_train_imbalanced = pd.read_csv('y test_processed data_GBP.csv')
y_train_imbalanced.columns = ['index','y']
y_train_imbalanced = y_train_imbalanced['y']


# Calculate baselines
baseline_train = (y_train.value_counts()/len(y_train))[0]
baseline_train

# Output: 0.05555555...

baseline_train_imbalanced = (y_train_imbalanced.value_counts()/len(y_train_imbalanced))[0]
baseline_train_imbalanced

# Output: 0.48042206...

baseline_test = (y_test.value_counts()/len(y_test))[0]
baseline_test

# Output: 0.48042206...

**Initial models**

The next stage is to try lots of models against the data and determine which model is worth investigating in more detail. This is determined by a couple of factors, including the accuracy score as well as the length of time the model takes to run.

In order to test each model, a cross validation was applied, which uses generated splits in the training data to determine how well the model will generalise to unseen data. The model was then fitted on the entire balanced training data and assessed against both the unbalanced training data and the test data. In total this provides three scores:
- cross validated score
- unbalanced training score
- test score

The cross validated score is used to determine between models which is the best model and the unbalanced score can be compared to the test score to confirm that the model generalises well.

Below is the code for the decision tree classifier. The code is identical for the other models, just with a different model defined (instead of dt_model = DecisionTreeClassifier()), so it is not reproduced here.

A summary of the models and scores is provided:

|Model<br>|Cross validated score<br>(baseline = 0.056)|Test data score<br>(baseline = 0.480)|Time taken for cross<br>validation (seconds)|
|---|---|---|---|
|Decision Tree|0.402|0.393|42.594|
|Logistic Regression|0.581|0.624|548.918|
|XG Boost|0.905|0.848|2824.799|
|Random Forest|0.962|0.937|213.555|
|Bagging (Decision tree)|0.959|0.936|1087.413|

In [ ]:
import time
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegressionCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

**Decision tree model**

Cross validation

In [ ]:
t0 = time.time()
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=10)
dt_scores = cross_val_score(dt_model, X_train, y_train, cv=skf, n_jobs=-1,verbose=1)
dt_scores
t1 = time.time()
print('Time taken: ',t1-t0)
print('Baseline: ',baseline_train)
print('Accuracy score: ',dt_scores.mean())

# Output:
# Time taken:  42.5940146446228
# Baseline:  0.05555555555555555
# Accuracy score:  0.4017963755481587

Score for test data

In [ ]:
dt_model.fit(X_train,y_train)
dt_test_predictions = dt_model.predict(X_test)
print('Baseline test: ',baseline_test)
print('Accuracy score: ',accuracy_score(y_test,dt_test_predictions))

# Output:
# Baseline test:  0.48042206382054625
# Accuracy score:  0.39279527860810914

**Further model refinement**

The next step is to determine which models to review in further detail and attempt to refine. As the random forest had the highest score and had a reasonable run time, this was the model chosen for further refinement.

There are limited parameters available for tuning a random forest. The main way to improve accuracy is by increasing the number of individual decision trees in the forest. However, the depth of each individual tree could be adjusted to determine if that increase the quality of prediction (although it is likely that a high depth will return the best result). In addition, the function used to measure the quality of a split can be adjusted (although this will likely have minimal impact).

In order to test multiple combinations of parameters for a model, a gridsearch can be applied, which iterates through each combination of parameters defined and calculates the score for each. The best combination of parameters and the resulting score can then be returned.

In [ ]:
from sklearn.model_selection import GridSearchCV

t0 = time.time()

rf_params = {
    'criterion':['gini','entropy'],
    'max_depth':[None,20,40,60]
}

rf_model = RandomForestClassifier(n_estimators=100,n_jobs=-1)

rf_gridsearch = GridSearchCV(rf_model,
                              rf_params,
                              n_jobs=-1, cv=5, verbose=1)

rf_gridsearch.fit(X_train, y_train)

t1 = time.time()
print('Time taken: ',t1-t0)
print('Score: ',rf_gridsearch.best_score_)
print('Best parameters: 'rf_gridsearch.best_params_)

The final model used was a Random Forest with 100 trees, each with no limit on the depth and using the gini criterion to measure the quality of each split in each tree. The outputs of the model (including the feature importances) were saved to .csv.

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,n_jobs=-1)
rf_model.fit(X_train,y_train)
rf_train_imbalanced_predictions = rf_model.predict(X_train_imbalanced)
print('Baseline - training imbalanced: ',baseline_train_imbalanced)
print('Accuracy - training imbalanced: ',accuracy_score(y_train_imbalanced,rf_train_imbalanced_predictions))

# Output:
# Baseline - training imbalanced:  0.48042206382054625
# Accuracy - training imbalanced:  0.9707978845711658

rf_test_predictions = rf_model.predict(X_test)
print('Baseline - test: ',baseline_test
print('Accuracy - test: ',accuracy_score(y_test,rf_test_predictions))

# Output:
# Baseline - test:  0.48042206382054625
# Accuracy - test:  0.9438747093840218
    
# Save outputs to .csv
pd.Series(rf_test_predictions).to_csv('Random forest predictions.csv')
y_test.to_csv('Actuals.csv')
pd.Series(rf_model.feature_importances_).to_csv('Random forest feature importance.csv')

### Evaluation

Clearly some model evaluation has occured already in order to determine which model to investigate in more detail.

However, model evaluation include more than reviewing the accuracy scores and the run time for the model. It also includes (for classification problems) reviewing the confusion matrix, the classification report, the ROC curve and calculating the area under the ROC curve (AUC-ROC).

In addition, reviewing the feature importances (if available) can provide additional information into what features the model is using for predictions. Comparing the feature importance across models can also help determine if the features are reasonably stable and likely to be true predictors or not.

**Confusion matrix and classification report**

The confusion matrix shows where the predicted classifications match the actual classificationsand, where they don't, the class that has been incorrectly predicted. It can be used to determine whether there are particular classes that the model consistently classifies incorrectly and then additional features or other changes could be implemented to attempt to rectify this.

In addition, there are other scores available via the classification report, including the precision, recall and f1 scores. In general, the closer to 1 that these values are then the better the model performs.

The class names have been changed to ensure sentitive data is not shown.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

# Import data
predictions = pd.read_csv('Random forest predictions.csv',names=['index','class'])
predictions = predictions['class']

actual = pd.read_csv('Actuals.csv',names=['index','class'])
actual = actual['class']

rf_feature_importance = pd.read_csv('Random forest feature importance.csv',names=['index','importance'])
rf_feature_importance = rf_feature_importance['importance']

bag_feature_importance = pd.read_csv('Bagging feature importance.csv',names=['index','importance'])
bag_feature_importance = bag_feature_importance['importance']

feature_names = pd.read_csv('X train_processed data_GBP_feature names.csv',names=['index','feature'])
feature_names = feature_names['feature']

Replace class names with labels

In [ ]:
# Replace class names with labels
actual_labels = list(set(actual))

replacement_dict = {j:'Label '+str(i+1) for i,j in enumerate(actual_labels)}

for key,value in replacement_dict.items():
    predictions = [text.replace(key,value) for text in predictions]
    actual = [text.replace(key,value) for text in actual]
    
con_mat_actual_labels = list(set(actual))
con_mat_pred_labels = ['Predicted '+i for i in con_mat_actual_labels]

Accuracy score

In [ ]:
print('Accuracy score: ',accuracy_score(actual,predictions))

# Output:
# Accuracy score:  0.9438747093840218

Confusion matrix

In [ ]:
# Show confusion matrix
conmat = np.array(confusion_matrix(actual, predictions, labels=con_mat_actual_labels))

confusion = pd.DataFrame(conmat, index=con_mat_actual_labels,
                         columns=con_mat_pred_labels)
confusion

In [1]:
# Output: Confusion matrix

,Unnamed: 0,Predicted Label 14,Predicted Label 13,Predicted Label 5,Predicted Label 6,Predicted Label 10,Predicted Label 4,Predicted Label 8,Predicted Label 7,Predicted Label 16,Predicted Label 9,Predicted Label 1,Predicted Label 17,Predicted Label 19,Predicted Label 15,Predicted Label 11,Predicted Label 12,Predicted Label 2,Predicted Label 18,Predicted Label 3
0,Label 14,6401,3,0,2,0,0,32,10,0,38,39,21,11,66,27,8,16,35,19
1,Label 13,3,10209,0,0,3,0,1,24,0,87,14,2,19,40,108,1,0,4,1
2,Label 5,2,0,657,0,0,0,0,0,0,0,1,1,0,1,1,0,0,4,3
3,Label 6,4,0,0,6087,0,0,0,2,0,1,3,1,8,50,4,0,4,1,9
4,Label 10,3,12,0,0,198,0,0,0,0,5,0,5,0,4,17,1,0,0,4
5,Label 4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,12
6,Label 8,6,0,0,0,0,0,1157,0,0,1,7,3,8,14,0,0,16,5,5
7,Label 7,1,52,0,0,1,0,0,5390,0,32,9,12,41,41,58,0,3,1,4
8,Label 16,1,0,0,0,0,0,0,0,127,0,0,0,0,1,0,0,0,0,0
9,Label 9,605,1448,0,0,10,0,17,393,0,88253,417,58,115,971,1371,83,32,131,117


Classification report

In [ ]:
print(classification_report(actual,predictions))

# Output:

#           precision    recall  f1-score   support

#      Label 1       0.96      0.96      0.96     17105
#     Label 10       0.92      0.80      0.85       249
#     Label 11       0.84      0.95      0.89     10605
#     Label 12       0.89      0.90      0.89      1240
#     Label 13       0.84      0.97      0.90     10516
#     Label 14       0.88      0.95      0.91      6728
#     Label 15       0.85      0.92      0.88     11774
#     Label 16       0.91      0.98      0.95       129
#     Label 17       0.92      0.93      0.92      5488
#     Label 18       0.95      0.95      0.95     11747
#     Label 19       0.85      0.86      0.85      1965
#      Label 2       0.95      0.97      0.96      8198
#      Label 3       0.84      0.90      0.87      2214
#      Label 4       0.00      0.00      0.00        15
#      Label 5       0.98      0.98      0.98       670
#      Label 6       0.98      0.99      0.98      6174
#      Label 7       0.90      0.95      0.93      5645
#      Label 8       0.90      0.95      0.93      1222
#      Label 9       1.00      0.94      0.97     94021

#    micro avg       0.94      0.94      0.94    195705
#    macro avg       0.86      0.89      0.87    195705
# weighted avg       0.95      0.94      0.94    195705

**Feature importance**

Reviewing the features that are used by the model to determine which features are most important when predicting the outcome is useful as you can identify potentially spurious predictions or unstable models. In addition, if multiple different models prioritise the same features then it is more likely they do have predictive ability.

This step reviewed the feature importance for the features used in both the Random Forest and the Bagging (decision tree) models to review if they are consistent. Again, the features have been changed to ensure sensitive data is not shown.

In [ ]:
# Create features dataframe
features = pd.DataFrame(feature_names,columns=['feature'])
features['rf_importance'] = rf_feature_importance
features['bag_importance'] = bag_feature_importance
features = features.set_index('feature')
features.sort_values(by='rf_importance',ascending=False,inplace=True)

In [ ]:
# Plot top 20 features for random forest against same 20 for bagging
number_of_features = 20
rf_feature_importance = features['rf_importance'][:number_of_features]
bag_feature_importance = features['bag_importance'][:number_of_features]
indices = range(len(rf_feature_importance))
names = [i.strip() for i in features[:number_of_features].index]
width = np.min(np.diff(indices))/3.

fig, ax = plt.subplots(1,1, figsize=(20,12))
ax.bar(indices,rf_feature_importance,width,color='b',label='Random Forest')
ax.bar(indices+width,bag_feature_importance,width,color='r',label='Bagging')
ax.set_xticks(indices+width/2)
ax.set_xticklabels(names)
plt.xticks(rotation=90)
plt.show()

**Output: **
Feature importance comparison for the top 20 features (as defined by the random forest model)

## Conclusion

Overall the model performs very well at predicting the correct classification for the invoice based on the information provided.

The confusion matrix shows there are relatively few consistent errors and the feature importances show that the different models are generally in agreement about which features are important. Therefore the model is likely to be stable.

This model is a good predictor of invoice classification.

## Considerations for real world production environments

In order to use this model in a production environment a number of additional developments and adaptations need to occur, including:
- developing a true proof of concept model that can apply to wider data than just invoices in GBP
- adapting the code so it is more structured and process orientated, rather than investigatory (including writin classes and functions)
- develop a standard pipeline for processing new data and running the model

In addition, continuous validation could be considered, to confirm that the model is still performing well. This could be implemented in a number of ways, including:
- continuing to compare the model predictions against manual classifications (as the manual process will still be in place for the majority of the business)
- compare the outputs of the supervised model with outputs of a clustering model and determining the level of agreement between the two models

Prior to deploying the model publicly, a true proof of concept could be developed, with a 'pickled' finalised model and, possibly, a nice user interface, potentially developed in Flask and hosted in a cloud environment (such as Azure or AWS).

---
# Appendices
---

## 1. Data dictionary

**Columns in database**

- Invoice Operating Unit
    - Client internal hierarchy
- Business Unit
    - Client internal hierarchy
- Service Line
    - Client internal hierarchy
- Region
    - Client internal hierarchy
- OU/Country
    - Client internal hierarchy
- Contract Cat
    - Used to derive category directly
    - Column will be excluded from dataset
- Supplier Number
- Supplier Name
- Invoice Number
    - Unique identifier - no predictive benefit so exclude
- Invoice Type
- Invoice Date
- Paid Date
- Invoice Desc
- PO Number
- Invoice_Amt
- Invoice Currency
- USD_Amt
- V Site Country
    - Client internal hierarchy
- Pay Group
    - Group of supplier?
- Payment Status
- Project Owning Org
    - Group owning the project
- Project Number
- Country
    - Client internal hierarchy
- datafile
    - Filename for source data
- datasource
    - Raw table for source data
- Legacy
    - Different parts of the group: AFW or WG
    - Derived from datasource
- Year
- Contract_type
    - Grouping of projects
    - Determined from project name
- Customer
    - Determined from project name
- Service_line
    - Determined from project name
- Funding_structure
    - Determined from project name
- InherentCategorization
    - Used to derive category directly
    - Column will be excluded from dataset
- ContractDerivedCategorization
    - Used to derive category directly
    - Column will be excluded from dataset
- SupplierDerivedCategorization
    - Used to derive category directly
    - Column will be excluded from dataset
- InherentCategorization_Group
    - Used to derive category directly
    - Column will be excluded from dataset
- ContractDerivedCategorization_Group
    - Used to derive category directly
    - Column will be excluded from dataset
- SupplierDerivedCategorization_Group
    - Used to derive category directly
    - Column will be excluded from dataset
- Category_Name
    - Sub classification of target
    - Column will be excluded from dataset
- Category_Group
    - Target
- Category_DirIndir
    - Derived from category
    - Column will be excluded from dataset
- Supplier_Group
    - Derived from Supplier Name
- Supplier_Sector
    - Derived from Supplier Name using existing fuzzy matching and GIS dataset
- Leakage_Identifier
    - Identifies purchases from competitors
- Leakage_Group
    - Competitor group identified from Leakage_Identifier
- Serv_Mtrl
    - Identifies if invoice is for Servies or Materials (based on PO number)
- Reimbursable_Flag
- Intercompany_Flag
    - Not interested in intercompany transactions
    - Intercompany transactions will be excluded from dataset
- CatOverwriteFlag
    - Flag to show if category is overwritten
    - Column will be excluded from dataset
- Americas_Flag
- OriginalCatName
    - Original category name
    - Column will be excluded from dataset
- Reporting_Date
- HistoricalCat_overwrite_Flag
    - Flag to show if category is overwritten
    - Column will be excluded from dataset